# day 19

In [ ]:
import re

## part 1

In [ ]:
def read(f):
    rules, inputs = open(f, "r").read().split("\n\n")
    inputs = " ".join(inputs.splitlines())
    return rules, inputs

In [ ]:
rules, inputs = read("./data/19_test.txt")
print(inputs)

This is a hack, but it will work much easier than any Python solution (like recursively generating regular expressions, etc).  By relying on `yacc`, I can generate a grammar from the given set of rules (which are helpfully given in a format very similar to `yacc`'s own):

In [ ]:
def gen_grammar(rules):
    grammar = ["start: r0 '\\n'; {printf(\"parsed\\n\"); exit(0);}"]
    for line in rules.splitlines():
        line = re.sub(r"(\d+)", r"r\1", line)
        grammar.append(line.replace('"', "'")+';')
    grammar = "\n".join(grammar)
    return grammar

In [ ]:
grammar = gen_grammar(rules)
print(grammar)

What's left is to plug them inside a skeleton `yacc` parser, which will exit at improper inputs, and print something (in this case, "parsed") when given correct ones.

```c
%{
#include<stdio.h>
#include<stdlib.h>
%}

%%

start : r0 '\n'            ; {printf("parsed\n"); exit(0);}
r0 : r4 r1 r5              ;
r1 : r2 r3 | r3 r2         ;
r2 : r4 r4 | r5 r5         ;
r3 : r4 r5 | r5 r4         ;
r4 : "a"                   ;
r5 : "b"                   ;

%%

void
yyerror(char const *s)
{
	exit(1);
}

int
yylex()
{
	char c = getchar();
	return c;
}

int
main()
{
	yyparse();
	return 0;
}

```

This can then be compiled with

    yacc ./test.y && cc -ll ./y.tab.c

And executed:

```sh
for i in ababbb bababa abbbab aaabbb aaaabbb
do
    echo $i | ./a.out
done | grep -c "parsed"

       2
```

Which passes for the test case.

Compiling with above values returns `235`.